In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
import pandas as pd
import matplotlib
import pandas_profiling
import os
print(os.getcwd())

D:\Project\Django\Scripts\server_project\test\owner_project\result


In [2]:
df19 = pd.read_excel("D:\Project\Data_set\data2019.xlsx")
df18 = pd.read_excel("D:\Project\Data_set\data2018.xlsx")

In [3]:
#백업용
df2018 = df18
df2019 = df19

In [4]:
# NULL값 제거
df19 = df19.dropna(axis=0)
df18 = df18.dropna(axis=0)
print(df19.shape)
print(df18.shape)

print(df18['상태'].value_counts())
print(df19['상태'].value_counts())

(326040, 27)
(206852, 27)
가입     97136
해약     85004
만기     10773
행사      9047
보류      2958
미계좌     1872
해지        62
Name: 상태, dtype: int64
가입     177062
해약     119423
만기      10747
행사      10364
보류       4469
해지       2558
미계좌      1340
멤버쉽        39
보류2        38
Name: 상태, dtype: int64


In [5]:
# 필요한 columns 
df18 = df18[['회원번호','성명','주민번호','주소','상태', '가입일자','최종불입일자','총납입회차','최종불입회차', '은행', '상품금액','총불입액','잔여금액','연체횟수','성별']]
df19 = df19[['회원번호','성명','주민번호','주소','상태', '가입일자','최종불입일자','총납입회차','최종불입회차', '은행', '상품금액','총불입액','잔여금액','연체횟수','성별']]

In [6]:
# '상태' 정리 2019년 데이터 
def transform_status(current_status, flag):
    if current_status == '가입':
        if flag == 0:
            return '만기'
        else:
            return current_status

    elif current_status == '해약':
        if flag == 0:
            return '만기_해약'
        else:
            return current_status

    elif current_status == '미계좌':
        return '해약'

    elif current_status == '보류':
        return '가입'

    elif current_status == '접수':
        return '가입'

    elif current_status == '보류2':
        return '가입'
    
    elif current_status == '멤버쉽':
        return '가입'
    
    elif current_status == '해지':
        return '해약'
    
    else:
        return current_status

In [7]:
# 상태는 (총납입회차 - 최종불입회차) 의 값을 기준으로 만기_해약 분류
df18['상태'] = df18.apply(lambda x: transform_status(x['상태'], x['총납입회차'] - x['최종불입회차']), axis=1)
df19['상태'] = df19.apply(lambda x: transform_status(x['상태'], x['총납입회차'] - x['최종불입회차']), axis=1)

In [8]:
def cut_year(day):
    return day[0:4]

df19['가입일자'] = df19['가입일자'].apply(lambda x: cut_year(x))
df19['최종불입일자'] = df19['최종불입일자'].apply(lambda x: cut_year(x))

In [11]:
# 가입 당시 나이
def age_converter(idnum, date):
    pre_age = (int(idnum[0:2]))
    s_age = int(date[2:4])
    if pre_age < 10:
        age = s_age - pre_age
    else:
        age = s_age - pre_age + 100
    return age

In [12]:
df19['가입나이'] = df19.apply(lambda x: age_converter(x['주민번호'], x['가입일자']), axis=1)
df19['해약나이'] = df19.apply(lambda x: age_converter(x['주민번호'], x['최종불입일자']), axis=1)

In [19]:
df19['가입나이'].describe()

count    326040.000000
mean         47.207652
std          14.337413
min           7.000000
25%          36.000000
50%          47.000000
75%          57.000000
max         109.000000
Name: 가입나이, dtype: float64

In [20]:
df19['해약나이'].describe()

count    326040.000000
mean         48.864323
std          14.599818
min           7.000000
25%          38.000000
50%          49.000000
75%          59.000000
max         109.000000
Name: 해약나이, dtype: float64

In [21]:
df19.head()

,회원번호,성명,주민번호,주소,상태,가입일자,최종불입일자,총납입회차,최종불입회차,은행,상품금액,총불입액,잔여금액,연체횟수,성별,가입나이,해약나이
0,1011A00017,박강석,480209-1000000,경기 고양시 덕양구 관산동 537-7 자성빌라 다동302,행사,2005,2010,100,100,농협단위조합,1980000,1980000,0,0,남,57,62
2,1011A00087,정병구,510129-1000000,경기 고양시 덕양구 행신2동 소만마을 616-804,행사,2005,2013,100,100,국민,1980000,1980000,0,0,남,54,62
3,1011A00097,이충일,550906-1000000,서울 마포구 망원1동 414-56,해약,2005,2005,100,1,하나(외환),1980000,19800,1960200,99,남,50,50
4,1011A00143,곽경근,611117-1000000,서울 마포구 망원1동 418-23 3층,해약,2005,2009,100,44,기업,1980000,871200,623700,56,남,44,48
7,1011A00219,박웅순,590410-1000000,충남 당진군 우강면 창리 201-5호,해약,2005,2006,100,4,농협단위조합,1980000,79400,1900600,96,남,46,47


In [22]:
# 전처리 함수들

# '현재 나이' 계산
def export_age(idnum):
    pre_age = (int(idnum[0:2]))
    if pre_age < 10:
        age = 20 - pre_age
    else:
        age = 120 - pre_age
    return age

# 사람 별 고유 id 뽑기
def transform_id(name, id_num):
    idnum = id_num[0:6]
    return name + idnum

# 연체횟수가 마이너스 인 값(행사값) 정리
def preprocess_event(num):
    if num < 0:
        return -1
    else:
        return num
    
# 진행률 = 최종불입회차 / 총 납입회차
def process_rate(total, last):
    rate = int(last) / int(total)
    return rate

# 성별 전처리
def preprocess_sex(sex):
    if sex =='남':
        return 0
    else: # 여자 1 
        return 1

# 앞에 두글자 가져와
def frontwo(s):
    return s[0:2]    

# 몇년차?
def year(plus, minus):
    start = str(plus)
    end = str(minus)
    y = int(end[0:4]) - int(start[0:4])
    return y

In [10]:
df18['이름번호'] = df18.apply(lambda x: transform_id(x['성명'], x['주민번호']), axis=1)

df18['주소'] = df18['주소'].apply(lambda x : frontwo(x))
df18['은행'] = df18['은행'].apply(lambda x : frontwo(x))
df18['가입연차'] = df18.apply(lambda x: year(x['가입일자'], x['최종불입일자']), axis=1)
df18['나이'] = df18['주민번호'].apply(lambda x : export_age(x))

# 연체횟수가 마이너스 인 값 정리 (모두 -1로 통일함)
df18['연체횟수'] = df18['연체횟수'].apply(lambda x : preprocess_event(x))
df18['진행률'] = df18.apply(lambda x: process_rate(x['총납입회차'] , x['최종불입회차']), axis=1)
df18['성별'] = df18['성별'].apply(lambda x : preprocess_sex(x))

In [24]:
df19.head(5)

,회원번호,성명,주민번호,주소,상태,가입일자,최종불입일자,총납입회차,최종불입회차,은행,...,총불입액,잔여금액,연체횟수,성별,가입나이,해약나이,이름번호,가입연차,나이,진행률
0,1011A00017,박강석,480209-1000000,경기,행사,2005,2010,100,100,농협,...,1980000,0,0,0,57,62,박강석480209,5,72,1.00
2,1011A00087,정병구,510129-1000000,경기,행사,2005,2013,100,100,국민,...,1980000,0,0,0,54,62,정병구510129,8,69,1.00
3,1011A00097,이충일,550906-1000000,서울,해약,2005,2005,100,1,하나,...,19800,1960200,99,0,50,50,이충일550906,0,65,0.01
4,1011A00143,곽경근,611117-1000000,서울,해약,2005,2009,100,44,기업,...,871200,623700,56,0,44,48,곽경근611117,4,59,0.44
7,1011A00219,박웅순,590410-1000000,충남,해약,2005,2006,100,4,농협,...,79400,1900600,96,0,46,47,박웅순590410,1,61,0.04


In [12]:
df18.to_csv("preprocess_2019.csv", index=False)

In [23]:
df19['이름번호'] = df19.apply(lambda x: transform_id(x['성명'], x['주민번호']), axis=1)

df19['주소'] = df19['주소'].apply(lambda x : frontwo(x))
df19['은행'] = df19['은행'].apply(lambda x : frontwo(x))
df19['가입연차'] = df19.apply(lambda x: year(x['가입일자'], x['최종불입일자']), axis=1)
df19['나이'] = df19['주민번호'].apply(lambda x : export_age(x))

# 연체횟수가 마이너스 인 값 정리 (모두 -1로 통일함)
df19['연체횟수'] = df19['연체횟수'].apply(lambda x : preprocess_event(x))
df19['진행률'] = df19.apply(lambda x: process_rate(x['총납입회차'] , x['최종불입회차']), axis=1)
df19['성별'] = df19['성별'].apply(lambda x : preprocess_sex(x))

In [25]:
df19.to_csv("preprocess_2019n.csv", index=False)

In [26]:
pr19 = df19.profile_report()
pr19.to_file("summary_report_2018.html")

interactions [continuous]:   1%|▍                                                      | 1/144 [00:00<00:58,  2.43it/s]D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 52509 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 45225 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 51077 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 54924 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 52264 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.

interactions [continuous]:   3%|█▉                                                     | 5/144 [00:01<00:55,  2.51it/s]D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 51092 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 50668 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:203: RuntimeWarning: Glyph 51092 missing from current font.
  glyph = font.load_char(ccode, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:203: RuntimeWarning: Glyph 50668 missing from current font.
  glyph = font.load_char(ccode, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 51092 missing from current font.
  font.set_text(s, 0.0, flags=flags)
D:\Project\Django\lib\site-packages

missing [bar]:   0%|                                                                             | 0/2 [00:00<?, ?it/s]D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 50896 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 48264 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 54840 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 49457 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.py:90: RuntimeWarning: Glyph 47749 missing from current font.
  font.set_text(s, 0.0, flags=LOAD_NO_HINTING)
D:\Project\Django\lib\site-packages\matplotlib\textpath.

build report structure: 100%|████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.14s/it]


In [18]:
df = df19[['주소', '상태', '가입일자', '최종불입일자','은행', '상품금액', '총불입액', '잔여금액', '연체횟수', '성별', '이름번호', '가입연차', '나이',
       '해약나이', '진행률']]

In [ ]:
pr19 = df.profile_report()
pr19.to_file("summary_report_2019.html")

variables:  94%|██████████████████████████████████████████████████████████████████▌    | 15/16 [00:18<00:00,  1.42it/s]